### Import all required libraries - numpy, pandas, json, requests, etc.

In [861]:
# The code was removed by Watson Studio for sharing.

In [862]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim


import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

from scipy import stats

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Scraping the wiki page using BeautifulSoup - import BeautifulSoup and install lxml

In [863]:
from bs4 import BeautifulSoup
!conda install -y lxml

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
lxml                      4.2.5            py35hefd8a0e_0  


### Get the url and print after scraping using BeautifulSoup - The URL is of the wiki page which provides the list of postal codes of Canada

In [864]:
webURL = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(webURL,'lxml')
#print(soup.prettify()) # tested and then commented only to not show entire o/p to maintain readability

### Find the table wikitable sortable which contains the postal codes and neighborhoods we require

In [865]:
postCodeTable = soup.find('table',{'class':'wikitable sortable'})
postCodeTable

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

### Getting the required data of the neighborhood, boroughs and postal code

In [866]:
postalCode = []
borough = []
neighborhood = []

for row in postCodeTable.find_all('tr')[1:]:  # skipping first row for header
    boroughCell = row.find_all('td')[1] # processing boroughs - second coloumn first to skip non assigned ones 
    if (boroughCell.text.strip() != 'Not assigned'):
        borough.append(boroughCell.text.strip())
        postCell = row.find_all('td')[0]  #  first coloumn
        postalCode.append(postCell.text.strip())
        neighborhoodCell = row.find_all('td')[2] #third coloumn
        if (neighborhoodCell.text.strip() == 'Not assigned'):
            neighborhood.append(boroughCell.text.strip())
        else:
            neighborhood.append(neighborhoodCell.text.strip())

### Load into a pandas df and using groupby and join get the final required output

In [867]:
df = pd.DataFrame()
df['PostalCode'] = postalCode
df['Borough'] = borough
df['Neighborhood'] = neighborhood
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### df.shape to print number of rows of df including header

In [868]:
df.shape[0]

212

In [869]:
!conda install -c conda-forge geocoder --yes

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geocoder                  1.38.1                     py_0    conda-forge


### Geocoder does not seem to work consistently and thus as suggested in the previous week using the CSV file of Geospatial data provided in the exercise

In [870]:
# geocoder latlong does not seem to work always, using the csv option instead

geospat_df = pd.read_csv('http://cocl.us/Geospatial_data')
#geospat_df.head()

In [871]:
df_toronto = df.set_index('PostalCode').join(geospat_df.set_index('Postal Code'), how = 'inner').reset_index()
df_toronto

,index,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [872]:
df_toronto.dtypes.index
#df_toronto.set_index('Neighborhood')

Index(['index', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

### Read the demographic data from the Toronto Open data neighborhood profile csv file. Refer data requirement documentation details provided.

In [873]:
#Read demographic data
path = 'https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df_toronto_demographics = pd.read_csv(path, encoding = "ISO-8859-1", engine='python')
df_toronto_demographics.head(5)

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

In [874]:
#Lets take the section which details of each neighborhood on the SouthAsian origin. Also we will include the total population row.
df_toronto_demo = df_toronto_demographics.iloc[np.r_[2,1579],:].copy()

In [875]:
df_toronto_demo.drop(df_toronto_demo.columns[[0,1,2,3,4]],axis=1,inplace=True)

In [876]:
df_toronto_demo

,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
2,"29,113","23,757","12,054","30,526","27,695","15,873","25,797","21,396","13,154","23,236","6,577","29,960","22,291","21,737","7,727","14,257","9,266","11,499","17,757","11,669","9,955","10,968","13,362","31,340","26,984","16,472","15,935","14,133","9,666","17,180","27,051","25,003","36,625","35,052","11,785","21,381","15,535","22,776","9,456","22,372","18,588","11,848","21,933","12,806","10,732","30,491","14,417","9,917","15,723","22,162","23,925","12,494","16,934","10,948","12,416","15,545","14,365","13,641","43,965","14,366","11,058","17,123","17,945","22,000","9,271","7,985","43,993","16,164","14,607","15,179","16,828","15,559","10,084","43,794","10,111","10,554","26,572","33,964","17,455","20,506","13,593","32,954","16,775","29,658","11,463","16,097","23,831","31,180","11,916","18,615","13,845","21,210","18,675","9,233","13,826","34,805","10,722","7,804","15,818","11,051","10,803","10,529","22,246","14,974","20,923","46,496","10,070","9,941","16,724","21,849","27,876","17,812","24,623","25,051","27,446","15,683","21,567","10,360","21,108","16,556","7,607","17,510","65,913","27,392","33,312","26,274","17,992","11,098","27,917","50,434","16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
1579,"5,260","4,170",575,"1,320","2,940",565,"2,515","1,410",805,535,410,"8,225","1,530","2,320",525,445,410,655,"1,245",525,275,355,"2,525","2,730","6,060",495,"2,995",390,760,"1,110","2,945","7,380","2,390","1,685",750,"1,760",400,"6,440",870,470,"1,340","1,220","7,460",265,280,"2,6

In [877]:
df_T = df_toronto_demo.T

In [878]:
df_T.reset_index()

,index,2,1579
0,Agincourt North,"29,113","5,260"
1,Agincourt South-Malvern West,"23,757","4,170"
2,Alderwood,"12,054",575
3,Annex,"30,526","1,320"
4,Banbury-Don Mills,"27,695","2,940"
5,Bathurst Manor,"15,873",565
6,Bay Street Corridor,"25,797","2,515"
7,Bayview Village,"21,396","1,410"
8,Bayview Woods-Steeles,"13,154",805
9,Bedford Park-Nortown,"23,236",535


### Relevant data for us is the South Asian population for each neighborhood

In [879]:
df_T.columns=['Total_population','South_Asian_Population']

In [880]:
#Merge this table with the post code and location tables
df_neighborhood = pd.merge(df_toronto, df_T, left_on='Neighborhood',right_index=True)

In [881]:
df_neighborhood

,index,Borough,Neighborhood,Latitude,Longitude,Total_population,South_Asian_Population
0,M1B,Scarborough,Rouge,43.806686,-79.194353,"46,496","20,265"
1,M1B,Scarborough,Malvern,43.806686,-79.194353,"43,794","18,085"
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497,"12,494","4,685"
5,M1E,Scarborough,Guildwood,43.763573,-79.188711,"9,917",970
6,M1E,Scarborough,Morningside,43.763573,-79.188711,"17,455","5,370"
7,M1E,Scarborough,West Hill,43.763573,-79.188711,"27,392","5,420"
8,M1G,Scarborough,Woburn,43.770992,-79.216917,"53,485","21,505"
10,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,"16,724","5,535"
12,M1K,Scarborough,Ionview,43.727929,-79.262029,"13,641","3,845"
13,M1K,Scarborough,Kennedy Park,43.727929,-79.262029,"17,123","4,360"


### To normalize this lets take % of South Asians to the population in the neighborhood

In [882]:
df_neighborhood.Total_population = df_neighborhood.Total_population.apply(lambda x: float(x.split()[0].replace(',', '')))
df_neighborhood.Total_population=df_neighborhood.Total_population.astype(float)
df_neighborhood.South_Asian_Population = df_neighborhood.South_Asian_Population.apply(lambda y: float(y.split()[0].replace(',', '')))
df_neighborhood.South_Asian_Population=df_neighborhood.South_Asian_Population.astype(float)

In [883]:
df_neighborhood['South_Asian_Percentage'] = (df_neighborhood['South_Asian_Population'] / df_neighborhood['Total_population'])*100

In [884]:
df_neighborhood.sort_values(['South_Asian_Percentage'], ascending=False, inplace=True)

In [885]:
df_neighborhood.reset_index(drop=True)

,index,Borough,Neighborhood,Latitude,Longitude,Total_population,South_Asian_Population,South_Asian_Percentage
0,M4H,East York,Thorncliffe Park,43.705369,-79.349372,21108.0,9545.0,45.219822
1,M1B,Scarborough,Rouge,43.806686,-79.194353,46496.0,20265.0,43.584394
2,M1B,Scarborough,Malvern,43.806686,-79.194353,43794.0,18085.0,41.295611
3,M1G,Scarborough,Woburn,43.770992,-79.216917,53485.0,21505.0,40.207535
4,M1C,Scarborough,Highland Creek,43.784535,-79.160497,12494.0,4685.0,37.497999
5,M1L,Scarborough,Oakridge,43.711112,-79.284577,13845.0,4750.0,34.308415
6,M3C,North York,Flemingdon Park,43.725900,-79.340923,21933.0,7460.0,34.012675
7,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,16724.0,5535.0,33.096149
8,M1E,Scarborough,Morningside,43.763573,-79.188711,17455.0,5370.0,30.764824
9,M1P,Scarborough,Dorset Park,43.757410,-79.273304,25003.0,7380.0,29.516458


### We have to create a priority rating i.e. preference rating for each neighborhood from customer which will be our profile for content based recommendation. We will assign priority based on South Asian population percentage

In [886]:
#Create a priority rating profile of neighborhood from the population parameter
dfPriorityNeighborhood = []
dfPriorityNeighborhood = pd.DataFrame(df_neighborhood, columns=['index','Borough','Neighborhood','Latitude','Longitude','Total_population','South_Asian_Population','South_Asian_Percentage','Priority'])
dfPriorityNeighborhood.drop(columns=['index','Borough','Latitude', 'Longitude', 'Total_population','South_Asian_Percentage','South_Asian_Population'], inplace=True)
dfPriorityNeighborhood.reset_index(drop=True, inplace=True)
#Assign a priortiy column for us to use it for recommendation - more the south asian population % higher is the priority i.e. preference given by the customer
dfPriorityNeighborhood['Priority']= '5'
dfPriorityNeighborhood['Priority'][10:15]='4'
dfPriorityNeighborhood['Priority'][15:20]='3'
dfPriorityNeighborhood['Priority'][20:25]='2'
dfPriorityNeighborhood['Priority'][25:]='1'

In [887]:
dfPriorityNeighborhood

,Neighborhood,Priority
0,Thorncliffe Park,5
1,Rouge,5
2,Malvern,5
3,Woburn,5
4,Highland Creek,5
5,Oakridge,5
6,Flemingdon Park,5
7,Scarborough Village,5
8,Morningside,5
9,Dorset Park,5


### Lets map Toronto and neighborhoods now

In [888]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [889]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_neighborhood['Latitude'], df_neighborhood['Longitude'], df_neighborhood['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Now lets get the best neigborhood based on both the above population data and some of foursquare inputs

### Exploring using Foursquare API

In [890]:
# The code was removed by Watson Studio for sharing.

### Lets find the places where there are Asian/Indian restaurants, food, shops and services so that there is a possibility of tie-ups with them for the store to deliver ther fresh requirements. Additionally we will look for the reachability for footfalls in these venues using Metros or Busstops. 

### First let us write a function to get details of venues within 500m radius in each of the above neighborhood

In [891]:
# Lets look at the fourt categories of interest i.e. pased on the problem we will look at the below venue categories and use search API
categories = {
    'Indian Restaurant': '4bf58dd8d48988d10f941735',
    'Shop & Service': '4d4b7105d754a06378d81259',
    'Bus Stop': '52f2ab2ebcbc57f1066b8b4f',
    'Metro Station': '4bf58dd8d48988d1fd931735'
}

In [892]:
', '.join([ category for category in categories])

'Bus Stop, Shop & Service, Metro Station, Indian Restaurant'

In [893]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [894]:
print(categories['Indian Restaurant'])
for category, category_id in categories.items():
    print(category)

4bf58dd8d48988d10f941735
Bus Stop
Shop & Service
Metro Station
Indian Restaurant


In [895]:
def getNearbyVenueDetails(Neighborhood_fn, Latitude_fn, Longitude_fn, category):
    radius=500 # limiting the radius to 500
    limit=30 #limiting the venues to 30 
    venues=[]

    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}&category={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                category,
                Latitude_fn, 
                Longitude_fn, 
                radius, 
                limit,
                categories[category])
                    
     
    # make the GET request
    results = requests.get(url).json()
        
    venues = results['response']['venues']
        
    # transform venues into a dataframe
    df = json_normalize(venues)
        
    cols = ['Venue','Venue Latitude','Venue Longitude']    
    if( len(df) == 0 ):        
        df = pd.DataFrame(columns=cols)
    else:        
        df = df[['name','location.lat','location.lng']]
        df.columns = cols
            
    return df

#### Fetching the venue details

In [ ]:
dfNearbyVenues=[]
dfNearbyVenues = pd.DataFrame(columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue Category',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude'])
dfFinalVenues=[]
dfFinalVenues = pd.DataFrame(columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue Category',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude'])

#Loop through
for (index, Borough, Neighborhood, Latitude, Longitude, Total_population, South_Asian_Population, South_Asian_Percentage) in df_neighborhood.itertuples(index=False):
        for category, category_id in categories.items():
            #Get the venue details
            dfNearbyVenues = getNearbyVenueDetails(Neighborhood, Latitude, Longitude, category)
            dfNearbyVenues['Neighborhood'] = Neighborhood
            dfNearbyVenues['Latitude'] = Latitude
            dfNearbyVenues['Longitude'] = Longitude
            dfNearbyVenues['Venue Category'] = category
            dfFinalVenues = dfFinalVenues.append(dfNearbyVenues)
        
#Saving this as maxing out daily limits while testing on calls to Foursqaure with free account
project.save_data(data=dfFinalVenues.to_csv(index=False),file_name='venues12.csv',overwrite=True)

In [896]:
#I have locally stored the foursquare fetched data else I will expire my API call limits on multiple runs. Lets read from previously explored and downloaded venue excel else API call limit is expiring.
#dfFinalVenues = pd.read_csv(project.get_file('venues12.csv'))

In [897]:
print('Total {} of venues are found'.format(len(dfFinalVenues)))

Total 282 of venues are found


In [898]:
dfFinalVenues.shape

(282, 7)

In [899]:
dfFinalVenues.head()

,Latitude,Longitude,Neighborhood,Venue,Venue Category,Venue Latitude,Venue Longitude
0,43.705369,-79.349372,Thorncliffe Park,Bus stop 8389,Bus Stop,43.702699,-79.346800
1,43.705369,-79.349372,Thorncliffe Park,TTC Stop #2979,Bus Stop,43.702430,-79.352345
2,43.705369,-79.349372,Thorncliffe Park,TTC Bus #100 Flemingdon Park,Bus Stop,43.707424,-79.348803
3,43.705369,-79.349372,Thorncliffe Park,25 Don Mills Bus Short Turn,Bus Stop,43.702521,-79.352660
4,43.705369,-79.349372,Thorncliffe Park,100 Flemingdon Park Bus Short-Turn,Bus Stop,43.701808,-79.353004


In [900]:
#Check number of veneues in each category in each neighborhood
dfFinalVenues['Venue Category'].value_counts().to_frame(name='Count')

,Count
Bus Stop,121
Shop & Service,66
Metro Station,48
Indian Restaurant,47


In [901]:
dfFinalVenues.shape

(282, 7)

### Data analysis and visualization

In [902]:
dfFinalVenueCnt = []
dfFinalVenueCnt = dfFinalVenues.groupby(['Neighborhood','Venue Category'], as_index=False).count()
dfFinalVenueCnt.shape

(81, 7)

In [903]:
dfFinalVenueCnt.dtypes

Neighborhood       object
Venue Category     object
Latitude            int64
Longitude           int64
Venue               int64
Venue Latitude      int64
Venue Longitude     int64
dtype: object

In [904]:
dfFinalVenueCnt.drop(columns=['Latitude','Longitude','Venue Latitude', 'Venue Longitude'], inplace=True)

In [905]:
dfFinalVenueCnt.rename(columns={'Venue':'Count'}, inplace=True)
dfFinalVenueCnt.shape

(81, 3)

In [906]:
dfFinalVenueCnt = dfFinalVenueCnt.pivot(index='Neighborhood', columns='Venue Category', values='Count').fillna(0)

In [907]:
dfFinalVenueCnt

Venue Category,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
Neighborhood,,,,
Agincourt North,0.0,2.0,1.0,0.0
Alderwood,2.0,2.0,0.0,2.0
Bathurst Manor,2.0,2.0,1.0,2.0
Dorset Park,2.0,2.0,0.0,3.0
Flemingdon Park,1.0,2.0,1.0,2.0
Forest Hill North,2.0,2.0,0.0,0.0
Guildwood,4.0,1.0,1.0,0.0
Henry Farm,11.0,0.0,5.0,8.0
Highland Creek,1.0,0.0,0.0,0.0


### Summary statistics of data

In [908]:
#Data summary
dfFinalVenueCnt.describe()

Venue Category,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
count,30.000000,30.000000,30.000000,30.000000
mean,4.033333,1.566667,1.600000,2.200000
std,6.048558,2.192201,1.811838,3.854241
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000,1.000000
75%,4.750000,2.000000,2.750000,2.000000
max,25.000000,9.000000,7.000000,18.000000


### Pearson correlation and p-values. We can see from this that Bus stop, India Restaurant and Shops and Service are highly corelated

In [909]:
#Finding correlations
dfFinalVenueCnt.corr()

Venue Category,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
Venue Category,,,,
Bus Stop,1.000000,0.781300,0.337936,0.875359
Indian Restaurant,0.781300,1.000000,-0.149324,0.802354
Metro Station,0.337936,-0.149324,1.000000,0.130361
Shop & Service,0.875359,0.802354,0.130361,1.000000


In [910]:
#p-values 
#Lets calculate p-value between Bus Stop, Indian Restaurant and Shop & Service

pearson_coef, p_value = stats.pearsonr(dfFinalVenueCnt['Bus Stop'], dfFinalVenueCnt['Indian Restaurant'])
print("The Pearson Correlation Coefficient b/w Bus Stop and Indian Restaurant is", pearson_coef, " with a P-value of P =", p_value)  

pearson_coef, p_value = stats.pearsonr(dfFinalVenueCnt['Bus Stop'], dfFinalVenueCnt['Shop & Service'])
print("The Pearson Correlation Coefficient b/w Bus Stop and Shop & Service is", pearson_coef, " with a P-value of P =", p_value)  

pearson_coef, p_value = stats.pearsonr(dfFinalVenueCnt['Shop & Service'], dfFinalVenueCnt['Indian Restaurant'])
print("The Pearson Correlation Coefficient b/w Shop & Service and Indian Restaurant", pearson_coef, " with a P-value of P =", p_value) 

The Pearson Correlation Coefficient b/w Bus Stop and Indian Restaurant is 0.781299938603  with a P-value of P = 3.47575062826e-07
The Pearson Correlation Coefficient b/w Bus Stop and Shop & Service is 0.875359085584  with a P-value of P = 2.45704084827e-10
The Pearson Correlation Coefficient b/w Shop & Service and Indian Restaurant 0.802354346792  with a P-value of P = 9.69636638855e-08


### Normalizing the Venue counts using min max scaler. Using this normalization of venue counts and the previous neighborhood priority profile (based on South Asian population) the intention is to find a weighted matrix of priority for each neighborhood. The highest priority will be the recommender output for location.

In [911]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [912]:
VenueNorm = scaler.fit_transform(dfFinalVenueCnt)

In [913]:
dfVenueNorm = pd.DataFrame(VenueNorm, index=dfFinalVenueCnt.index,)

In [914]:
dfVenueNorm.columns=['Bus Stop', 'Indian Restaurant', 'Metro Station', 'Shop & Service']

In [915]:
dfVenueNorm.reset_index(inplace=True)

### Normalized Venue counts for each neighborhood

In [916]:
dfVenueNorm

,Neighborhood,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
0,Agincourt North,0.00,0.222222,0.142857,0.000000
1,Alderwood,0.08,0.222222,0.000000,0.111111
2,Bathurst Manor,0.08,0.222222,0.142857,0.111111
3,Dorset Park,0.08,0.222222,0.000000,0.166667
4,Flemingdon Park,0.04,0.222222,0.142857,0.111111
5,Forest Hill North,0.08,0.222222,0.000000,0.000000
6,Guildwood,0.16,0.111111,0.142857,0.000000
7,Henry Farm,0.44,0.000000,0.714286,0.444444
8,Highland Creek,0.04,0.000000,0.000000,0.000000
9,Humber Summit,0.00,0.111111,0.142857,0.000000


In [917]:
Priority = []

In [918]:
dfPriorityNeighborhood.set_index("Neighborhood")
for i, row in enumerate(dfVenueNorm.itertuples()):
    for key, row in enumerate (dfPriorityNeighborhood.itertuples()):
        if dfVenueNorm["Neighborhood"][i] == dfPriorityNeighborhood["Neighborhood"][key]:
           Priority.append({'Neighborhood':dfVenueNorm["Neighborhood"][i],
                              'Priority':dfPriorityNeighborhood["Priority"][key]})
dfPriority = pd.DataFrame(Priority, columns=['Neighborhood','Priority'])
dfPriorityNeighborhood.set_index("Neighborhood")
dfPriority

,Neighborhood,Priority
0,Agincourt North,3
1,Alderwood,1
2,Bathurst Manor,1
3,Dorset Park,5
4,Flemingdon Park,5
5,Forest Hill North,1
6,Guildwood,2
7,Henry Farm,4
8,Highland Creek,5
9,Humber Summit,4


### Getting all data in one table - both reference profile of priorities and normalized venue counts

In [919]:
venuePriorities = pd.merge(dfPriority, dfVenueNorm)

In [920]:
venuePriorities

,Neighborhood,Priority,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
0,Agincourt North,3,0.00,0.222222,0.142857,0.000000
1,Alderwood,1,0.08,0.222222,0.000000,0.111111
2,Bathurst Manor,1,0.08,0.222222,0.142857,0.111111
3,Dorset Park,5,0.08,0.222222,0.000000,0.166667
4,Flemingdon Park,5,0.04,0.222222,0.142857,0.111111
5,Forest Hill North,1,0.08,0.222222,0.000000,0.000000
6,Guildwood,2,0.16,0.111111,0.142857,0.000000
7,Henry Farm,4,0.44,0.000000,0.714286,0.444444
8,Highland Creek,5,0.04,0.000000,0.000000,0.000000
9,Humber Summit,4,0.00,0.111111,0.142857,0.000000


In [921]:
#Align it 
venuePriorities.drop(['Neighborhood','Priority'], axis=1, inplace=True)
venuePriorities

,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
0,0.00,0.222222,0.142857,0.000000
1,0.08,0.222222,0.000000,0.111111
2,0.08,0.222222,0.142857,0.111111
3,0.08,0.222222,0.000000,0.166667
4,0.04,0.222222,0.142857,0.111111
5,0.08,0.222222,0.000000,0.000000
6,0.16,0.111111,0.142857,0.000000
7,0.44,0.000000,0.714286,0.444444
8,0.04,0.000000,0.000000,0.000000
9,0.00,0.111111,0.142857,0.000000


In [922]:
venuePriorities.shape

(30, 4)

In [923]:
dfPriority.shape

(30, 2)

In [924]:
dfPriority = dfPriority.astype({"Priority": float})

In [925]:
venueMatrix = venuePriorities.transpose().dot(dfPriority['Priority'])

In [926]:
venueMatrix

Bus Stop             14.440000
Indian Restaurant    12.666667
Metro Station        26.857143
Shop & Service        9.000000
dtype: float64

### Recommendation based on the final weighted average priority

In [927]:
dfReco = (venuePriorities*venueMatrix).sum(axis=1)/venueMatrix.sum()
dfReco = dfReco.reset_index(name='Priority')

In [928]:
dfReco.sort_values(by='Priority', ascending=False)

,index,Priority
21,21,0.678318
12,12,0.676975
20,20,0.507878
7,7,0.469115
10,10,0.297551
11,11,0.297551
14,14,0.243742
23,23,0.243742
22,22,0.207742
27,27,0.201154


### Regent Park neighborhood is with maximum priority or rating and is thus recommended as the final location.

In [929]:
### Thus we recommend Regent Park (max prority) neighborhood 
dfReco.max()

index       29.000000
Priority     0.678318
dtype: float64